In [8]:
import requests
import json


betano_url_game = "https://br.betano.com/odds/detroit-pistons-sacramento-kings/43722583/"
def get_content():
    r = requests.get(betano_url_game)
    content = r.text
    return content
def get_bet_event():
    content = get_content()
    token_mark = '<script>window["initial_state"]='
    inx_start = content.find(token_mark) + len(token_mark)
    inx_end = content.find('</script>',inx_start)
    json_content = content[inx_start:inx_end]
    dic_data = json.loads( json_content)
    bet_event  = dic_data["data"]["event"]
    return bet_event

def get_players(event):
    
    match = event["name"]
    separator = ' - '
    player1, player2 = str.split(match, separator) 
    return player1, player2


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"]==value, lst))

def get_bet_target(token_class, bet_player, markets ):

    winner_markets = find_by_name(token_class, markets)
    if(len(winner_markets)>0):
        winner_market = winner_markets[0]
    else:
        raise Exception(f"ERROR: {token_class} not found")

    tracked_player_list  =find_by_name(bet_player,winner_market["selections"])
    if(len(tracked_player_list)>0):
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"]==value, lst))

def get_bet_target(token_class, bet_player, markets ):

    winner_markets = find_by_name(token_class, markets)
    if(len(winner_markets)>0):
        winner_market = winner_markets[0]
    else:
        raise Exception(f"ERROR: {token_class} not found")

    tracked_player_list  =find_by_name(bet_player,winner_market["selections"])
    if(len(tracked_player_list)>0):
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj

def has_triggered_bet(tracked_player_obj, bet_value_expected):
    if(tracked_player_obj["price"]>= bet_value_expected):
        msg = f'{tracked_player_obj["name"]} alcancou {bet_value_expected}: Atual {tracked_player_obj["price"]}'
        return msg
    
def is_second_half(event):
    period = event["liveData"]["periodDescription"]
    if period != "P1" and period!="P2":
        return True
def can_bet_for_total_points_on_2nd_half(event):

    if is_second_half (event):
        first = event["liveData"]["results"]["firstPeriodScore"]
        second = event["liveData"]["results"]["secondPeriodScore"]
        firstHome,firstAway = first["home"],first["away"]
        secondHome,secondAway = second["home"],second["away"]
        totalFirstHalf = int(firstHome)+int(firstAway)+int(secondHome)+int(secondAway)
        markets = event["markets"]
        total_points = find_by_name("Total de Pontos", markets)
        less_than_points_obj = total_points[0]["selections"][1]
        handicap = less_than_points_obj["handicap"]
        projectedPoints = totalFirstHalf*2
        can_bet = "NO" if projectedPoints-3 >= handicap else "YES"
        print(f"IT'S SECOND HALF - TOTAL POINTS: {totalFirstHalf}*2={projectedPoints} - hand: {handicap} -> can bet? {can_bet}")
        if can_bet=="YES":
            return "APOSTAR EM TOTAL DE PONTOS"

In [9]:
import time
from plyer import notification
import pygame
import ctypes  # An included library with Python install.
import winsound

# Inicializa o mixer do pygame para sons
pygame.mixer.init()

def enviar_alerta(msg):
    # Exibe um popup
    # notification.notify(
    #     title='Alerta',
    #     message=msg,
    #     app_name='Temporizador'
    # )
    
    Mbox.BoxOkCancelAsync("ALERTA BET", msg)

    # Toca um som de alerta
    pygame.mixer.music.load('alarm.mp3')
    pygame.mixer.music.play()


class Mbox:
    OK = 1
    CANCELADO = 2

    def BoxOkCancel(title, text):
        winsound.MessageBeep(winsound.MB_OK)
        ctypes.windll.user32.MessageBoxW(0, text, title, 1)

    def Alerta(title, text=""):
        winsound.MessageBeep(winsound.MB_OK)
        funcao = lambda: ctypes.windll.user32.MessageBoxW(0, text, title, 0x1000)
        Mbox.executar_alerta_paralelo(funcao)

    def executar_alerta_paralelo(funcao_alerta, *args):
        thread_exec = ThreadSimples(funcao_alerta, *args)
        thread_exec.start()

    def BoxOkCancelAsync(title, text):
        Mbox.executar_alerta_paralelo(Mbox.BoxOkCancel, title, text)


In [10]:
import threading
import time
TEMPO_ALARME   =  30 # SEGUNDOS
class ThreadCustom(threading.Thread):
    def __init__(self, tempo, executar_funcao):
        super(ThreadCustom, self).__init__()
        self.parar = threading.Event()
        self.tempo = tempo
        self.executar = executar_funcao

    def run(self):
        while not self.parar.is_set():
            self.executar()
            time.sleep(self.tempo)

    def parar_execucao(self):
        print("Parando a execução...")
        self.parar.set()

def parar_thread(thread):
    thread.parar_execucao()
    thread.join()  # Aguarde até que a thread termine completamente

class ThreadSimples(threading.Thread):
    def __init__(self, funcao, *args, **kwargs):
        # If you are going to have your own constructor, make sure you call the parent constructor too!
        #        Thread.__init__(self)
        super().__init__()
        self.funcao = funcao
        self.kwargs = kwargs
        self.args = args

    def run(self):
        self.funcao(*self.args, **self.kwargs)

In [11]:

def main():
    event = get_bet_event()
    player1, player2 = get_players(event)
    tracked_player = player2
    bet_value_expected = 3
    markets = event["markets"]
    token_winner = "Vencedor"

    tracked_player_obj = get_bet_target(token_winner, tracked_player, markets)
    print(f'{token_winner}:{tracked_player_obj["name"]}:{tracked_player_obj["price"]}')
    result = has_triggered_bet(tracked_player_obj, bet_value_expected)
    if result:
        enviar_alerta(result)

    bet_on_2nd_half = can_bet_for_total_points_on_2nd_half(event)
    if bet_on_2nd_half:
        enviar_alerta(bet_on_2nd_half)
# main()

In [12]:

thread = ThreadCustom(TEMPO_ALARME,main)
thread.start()

In [13]:
# parar_thread(thread)

Vencedor:Sacramento Kings:1.26
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 249.5 -> can bet? NO
Vencedor:Sacramento Kings:1.31
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.37
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.2
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.11
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 251.5 -> can bet? NO
Vencedor:Sacramento Kings:1.1
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 249.5 -> can bet? NO
Vencedor:Sacramento Kings:1.11
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.11
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.1
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.1
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - h

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11216\3542817686.py", line 13, in run
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11216\2787219941.py", line 15, in main
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11216\3992361451.py", line 84, in can_bet_for_total_points_on_2nd_half
IndexError: list index out of range


Vencedor:Sacramento Kings:1.007
